In [1]:
from ipstartup import *
from mapboxer.elections import get

[root:32:INFO]:logging started from c:\users\simon\onedrive\documents\py\live\utils\defaultlog.py (time=Aug-12 20:46 MainProcess)


# get data

In [5]:
allres = []
for year in range(2010, 2016):
    allres.append(get.res_local(year))
allres = pd.concat(allres)
allres.head()

[cellevents:43:INFO]:starting (time=Aug-12 20:46 MainProcess)


,authority,wardname,year,party,votes
0,Adur,Buckingham,2010,C,1301
1,Adur,Buckingham,2010,LD,828
2,Adur,Buckingham,2010,UKIP,197
3,Adur,Churchill,2010,C,885
4,Adur,Churchill,2010,LD,762


time: 551 ms


# tables

In [6]:
df = allres.copy()
df.loc[~df.party.isin(["C", "LD", "UKIP", "Lab", "Grn"]), "party"] = "other"
cands = pd.crosstab(df.party, df.year, df.wardname, aggfunc="count")
cands["total"] = cands.sum(axis=1)
d("candidates")
cands.style.format("{:,}")

[cellevents:43:INFO]:starting (time=Aug-12 20:46 MainProcess)


'candidates'

year,2010,2011,2012,2013,2014,2015,total
party,,,,,,,
C,"4,204","8,843","3,274","2,274","4,149","8,670","31,414"
Grn,"1,645","1,590","1,017",890,"1,875","3,538","10,555"
LD,"3,494","5,608","2,278","1,761","2,935","4,300","20,376"
Lab,"4,019","6,810","3,685","2,200","4,195","6,995","27,904"
UKIP,626,"1,230",744,"1,738","2,196","4,098","10,632"
other,"2,027","3,061","3,616","1,260","1,618","2,985","14,567"


time: 195 ms


In [7]:
missing = cands.max(axis=0)-cands
d("missing candidates")
missing.style.format("{:,}")

[cellevents:43:INFO]:starting (time=Aug-12 20:46 MainProcess)
[numexpr.utils:141:INFO]:NumExpr defaulting to 8 threads. (time=Aug-12 20:46 MainProcess)


'missing candidates'

year,2010,2011,2012,2013,2014,2015,total
party,,,,,,,
C,0,0,411,0,46,0,0
Grn,"2,559","7,253","2,668","1,384","2,320","5,132","20,859"
LD,710,"3,235","1,407",513,"1,260","4,370","11,038"
Lab,185,"2,033",0,74,0,"1,675","3,510"
UKIP,"3,578","7,613","2,941",536,"1,999","4,572","20,782"
other,"2,177","5,782",69,"1,014","2,577","5,685","16,847"


time: 44 ms


In [8]:
d("missing candidates%")
(missing/cands.max(axis=0)).style.format("{:.0%}")

[cellevents:43:INFO]:starting (time=Aug-12 20:47 MainProcess)


'missing candidates%'

year,2010,2011,2012,2013,2014,2015,total
party,,,,,,,
C,0%,0%,11%,0%,1%,0%,0%
Grn,61%,82%,72%,61%,55%,59%,66%
LD,17%,37%,38%,23%,30%,50%,35%
Lab,4%,23%,0%,3%,0%,19%,11%
UKIP,85%,86%,80%,24%,48%,53%,66%
other,52%,65%,2%,45%,61%,66%,54%


time: 17 ms


In [9]:
votes = pd.crosstab(df.party, df.year, df.votes, aggfunc="sum")
votes["total"] = votes.sum(axis=1)
d("vote share")
(votes/votes.sum(axis=0)).style.format("{:.0%}")

[cellevents:43:INFO]:starting (time=Aug-12 20:47 MainProcess)


'vote share'

year,2010,2011,2012,2013,2014,2015,total
party,,,,,,,
C,34%,41%,23%,34%,28%,39%,35%
Grn,3%,3%,3%,3%,6%,6%,4%
LD,23%,16%,11%,14%,11%,10%,15%
Lab,33%,30%,41%,22%,39%,28%,32%
UKIP,1%,2%,3%,19%,11%,11%,7%
other,6%,8%,19%,8%,5%,6%,7%


time: 53 ms


In [10]:
d("votes/candidate")
(votes/cands).style.format("{:,.0f}")

[cellevents:43:INFO]:starting (time=Aug-12 20:47 MainProcess)


'votes/candidate'

year,2010,2011,2012,2013,2014,2015,total
party,,,,,,,
C,"1,588",825,574,930,825,"1,271","1,031"
Grn,416,340,255,242,386,445,379
LD,"1,313",517,410,486,457,669,662
Lab,"1,650",790,898,637,"1,124","1,117","1,048"
UKIP,365,266,306,706,629,744,606
other,549,437,431,385,347,538,458


time: 10 ms


In [14]:
df[(df.year==2010) & (df.party=="LD")]

[cellevents:43:INFO]:starting (time=Aug-12 20:51 MainProcess)


,authority,wardname,year,party,votes
1,Adur,Buckingham,2010,LD,828
4,Adur,Churchill,2010,LD,762
8,Adur,Cokeham,2010,LD,594
12,Adur,Eastbrook,2010,LD,711
18,Adur,Hillside,2010,LD,667
...,...,...,...,...,...
15981,Wyre Forest,Franche,2010,LD,446
15990,Wyre Forest,Habberley and Blakebrook,2010,LD,458
16004,Wyre Forest,Oldington and Foley Park,2010,LD,378
16007,Wyre Forest,Sutton Park,2010,LD,612


time: 29 ms


In [13]:
df[(df.year==2015) & (df.party=="LD")] # 4300

[cellevents:43:INFO]:starting (time=Aug-12 20:50 MainProcess)


,authority,wardname,year,party,votes
26,Allerdale,Christchurch,2015,LD,193
32,Allerdale,Dalton,2015,LD,226
60,Allerdale,Harrington,2015,LD,176
69,Allerdale,Keswick,2015,LD,661
92,Allerdale,St Johns,2015,LD,425
...,...,...,...,...,...
30564,York,Strensall,2015,LD,775
30569,York,Westfield,2015,LD,2866
30570,York,Westfield,2015,LD,2369
30571,York,Westfield,2015,LD,2106


time: 26 ms


# ratio

In [ ]:
# ratio by year
df = allres.copy()
df = pd.crosstab(
    [df.authority, df.wardname, df.year], df.party, df.votes, aggfunc="sum"
).reset_index()
df.columns.name = ""
df = df.fillna(0)
df["ratio"] = df.LD / df.drop(["LD", "year"], axis=1).max(axis=1)
df.ratio = df.ratio * 100

In [ ]:
# by year
byyear = pd.crosstab([df.authority, df.wardname], df.year, df.ratio, aggfunc="first").reset_index()
byyear[byyear.authority=="Stratford-on-Avon"]